In [67]:
import numpy as np
import scipy as sp
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import sklearn as sk

import matplotlib as mpl
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
sns.set()
sns.set_style("whitegrid")
sns.set_color_codes()

In [1]:
# 경로 변수 (고정)

dataLoc = '../_data/'

trainfile = 'train_03_997.csv'
old_trainfile = 'train.csv'
kaggle_trainfile = 'train_k.csv'

testfile = 'test_clean.csv'
old_testfile = 'test.csv'

train_171016 = '171016_dow_hr_pas_L2_dur_only.csv'
train_osrm = '../_data/train_with_osrm.csv'
train_osrm_weather = '../_data/train_with_osrm_and_weather.csv'
train_osrm_weather_hourly = '../_data/train_with_osrm_and_weather_hourly.csv'
train_dropped_nan_fixed = '../_data/train__dropped__no_disaster_nan_fixed.csv'
train_added_airps = '../_data/train__dropped__lnglat_saved__airports__no_disaster.csv' #701778
train_added_airps_cleaned = '../_data/train__dropped__cleaned__airports__no_disaster.csv'
train_added_airps_disas_cleaned = '../_data/train__dropped__cleaned__eval__airports__disaster.csv'
train_added_weekhr = 'train__dropped__cleaned__eval__a_d_weekhour.csv'
train_added_manh = '../_data/train__dropped__air_bliz_weekhr_manh.csv'
train_all_vars_rough = '../_data/train__all_vars__rough.csv'
train_701778_all_vars = '../_data/train__701778__all_vars.csv'
train_latest = train_added_airps #7

In [5]:
pd.options.display.max_columns = 40

to insert:
  * is_hol
  * m_d
  * dow
  * hr
  * L2
  * airps
  * total_dist, total_time
  * day weather
  * hr weather

In [3]:
df_train.columns

Index(['is_hol', 'blizz', 'm_d', 'dow', 'hr', 'wkhr', 'L2', 'pas_cnt', 'p_JFK',
       'p_LGA', 'd_JFK', 'd_LGA', 'p_lng', 'p_lat', 'd_lng', 'd_lat',
       'total_dist', 'total_time', 'day_max_temp', 'day_min_temp',
       'day_rainfall', 'day_snowfall', 'day_snowdepth', 'hr_temp', 'hr_humid',
       'hr_dewpt', 'hr_windspeed', 'hr_gustspeed', 'hr_precip', 'dur'],
      dtype='object')

```
move dur to end
weekhour
blizz
```

In [2]:
# train_with_osrm_and_weather.csv
df_train = pd.read_csv(dataLoc + train_701778_all_vars)
df_train.head()

,is_hol,blizz,m_d,dow,hr,wkhr,L2,pas_cnt,p_JFK,p_LGA,...,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip,dur
0,False,0,430,Saturday,23,143,0.069424,1,False,False,...,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1454
1,False,0,430,Saturday,23,143,0.057791,1,False,False,...,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1409
2,False,0,430,Saturday,23,143,0.066698,2,False,False,...,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1081
3,False,0,430,Saturday,23,143,0.018223,1,False,False,...,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,800
4,False,0,430,Saturday,23,143,0.027198,1,False,False,...,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1151


move duration to end

In [8]:
df_train = pd.concat([df_train.drop('dur', axis=1), df_train['dur']], axis=1)
df_train.head()

,is_hol,m_d,dow,hr,L2,pas_cnt,p_JFK,p_LGA,d_JFK,d_LGA,p_lng,p_lat,d_lng,d_lat,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip,dur
0,False,430,Saturday,23,0.069424,1,False,False,False,False,-73.987793,40.724792,-73.975616,40.656445,10010.9,861.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1454
1,False,430,Saturday,23,0.057791,1,False,False,False,False,-73.957596,40.717770,-73.951424,40.775230,10357.0,786.3,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1409
2,False,430,Saturday,23,0.066698,2,False,False,False,False,-74.000954,40.742031,-73.947708,40.782200,9275.8,703.9,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1081
3,False,430,Saturday,23,0.018223,1,False,False,False,False,-73.985733,40.738258,-73.993179,40.754890,2726.9,298.0,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,800
4,False,430,Saturday,23,0.027198,1,False,False,False,False,-74.006615,40.740650,-73.985619,40.723362,3455.8,458.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1151


---
add weekhour

In [11]:
# init weekdays list

weekdays = list()
for i in range(7):
    time_ = pd.Timestamp(2017, 10, 16+i, 12, 0, 0)
    weekdays.append(time_.weekday_name)

weekdays

['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [9]:
def get_weekhour(dow, hr):
    return weekdays.index(dow)*24 + hr

In [13]:
n = 10000
pd.concat([df_train[['dow', 'hr']],\
           pd.Series(np.vectorize(get_weekhour)(df_train['dow'], df_train['hr']), name='weekhour')],
          axis=1)[n:n+10]

,dow,hr,weekhour
10000,Friday,13,109
10001,Friday,13,109
10002,Friday,13,109
10003,Friday,13,109
10004,Friday,13,109
10005,Friday,13,109
10006,Friday,13,109
10007,Friday,13,109
10008,Friday,13,109
10009,Friday,13,109


In [16]:
df_copy = df_train.copy()

In [17]:
df_copy.insert(4, 'wkhr', pd.Series(np.vectorize(get_weekhour)(df_train['dow'], df_train['hr']), name='weekhour'))
df_copy.head()

,is_hol,m_d,dow,hr,wkhr,L2,pas_cnt,p_JFK,p_LGA,d_JFK,d_LGA,p_lng,p_lat,d_lng,d_lat,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip,dur
0,False,430,Saturday,23,143,0.069424,1,False,False,False,False,-73.987793,40.724792,-73.975616,40.656445,10010.9,861.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1454
1,False,430,Saturday,23,143,0.057791,1,False,False,False,False,-73.957596,40.717770,-73.951424,40.775230,10357.0,786.3,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1409
2,False,430,Saturday,23,143,0.066698,2,False,False,False,False,-74.000954,40.742031,-73.947708,40.782200,9275.8,703.9,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1081
3,False,430,Saturday,23,143,0.018223,1,False,False,False,False,-73.985733,40.738258,-73.993179,40.754890,2726.9,298.0,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,800
4,False,430,Saturday,23,143,0.027198,1,False,False,False,False,-74.006615,40.740650,-73.985619,40.723362,3455.8,458.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1151


### todo: add `blizz` var
* 1/23 : 6
* 1/24 : 5
* 1/25 : 4
* 1/26 : 3
* 1/27 : 2
* 1/28 : 1
* others : 0

In [18]:
df_train['m_d'][df_train['m_d'] == 123]

577829    123
577830    123
577831    123
577832    123
577833    123
577834    123
577835    123
577836    123
577837    123
577838    123
577839    123
577840    123
577841    123
577842    123
577843    123
577844    123
577845    123
577846    123
577847    123
577848    123
577849    123
577850    123
577851    123
577852    123
577853    123
577854    123
577855    123
577856    123
577857    123
577858    123
         ... 
578994    123
578995    123
578996    123
578997    123
578998    123
578999    123
579000    123
579001    123
579002    123
579003    123
579004    123
579005    123
579006    123
579007    123
579008    123
579009    123
579010    123
579011    123
579012    123
579013    123
579014    123
579015    123
579016    123
579017    123
579018    123
579019    123
579020    123
579021    123
579022    123
579023    123
Name: m_d, Length: 1195, dtype: int64

In [21]:
def get_blizz(md):
    m = int(md[0])
    d = int(md[1:])
    blizz = 0
    if m == 1 and 23 <= d <= 28:
        blizz = 29 - d
    return blizz

df_copy.insert(1, 'blizz', df_train.loc[:, 'm_d'].apply(str).apply(get_blizz))

In [23]:
df_copy.loc[124 == df_train['m_d']].head()

,is_hol,blizz,m_d,dow,hr,wkhr,L2,pas_cnt,p_JFK,p_LGA,d_JFK,d_LGA,p_lng,p_lat,d_lng,d_lat,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip,dur
575399,False,5,124,Sunday,23,167,0.069620,1,False,True,False,False,-73.863052,40.769279,-73.796631,40.790138,10307.5,756.3,35,20,0.0,0.0,22.0,0.0,0.45,-10.6,9.3,0.0,0.0,1557
575400,False,5,124,Sunday,23,167,0.042868,1,False,False,False,False,-73.960403,40.761612,-73.921616,40.743359,4771.7,332.3,35,20,0.0,0.0,22.0,0.0,0.45,-10.6,9.3,0.0,0.0,637
575401,False,5,124,Sunday,23,167,0.028081,5,False,False,False,False,-73.948738,40.781879,-73.965340,40.759232,2881.2,233.6,35,20,0.0,0.0,22.0,0.0,0.45,-10.6,9.3,0.0,0.0,361
575402,False,5,124,Sunday,23,167,0.019552,1,False,False,False,False,-73.953217,40.775810,-73.967766,40.762749,2237.6,185.2,35,20,0.0,0.0,22.0,0.0,0.45,-10.6,9.3,0.0,0.0,357
575403,False,5,124,Sunday,23,167,0.004694,1,False,False,False,False,-73.954590,40.765598,-73.954193,40.770275,689.4,60.7,35,20,0.0,0.0,22.0,0.0,0.45,-10.6,9.3,0.0,0.0,221


In [25]:
df_copy.shape

(701778, 30)

In [26]:
df_copy.head()

,is_hol,blizz,m_d,dow,hr,wkhr,L2,pas_cnt,p_JFK,p_LGA,d_JFK,d_LGA,p_lng,p_lat,d_lng,d_lat,total_dist,total_time,day_max_temp,day_min_temp,day_rainfall,day_snowfall,day_snowdepth,hr_temp,hr_humid,hr_dewpt,hr_windspeed,hr_gustspeed,hr_precip,dur
0,False,0,430,Saturday,23,143,0.069424,1,False,False,False,False,-73.987793,40.724792,-73.975616,40.656445,10010.9,861.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1454
1,False,0,430,Saturday,23,143,0.057791,1,False,False,False,False,-73.957596,40.717770,-73.951424,40.775230,10357.0,786.3,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1409
2,False,0,430,Saturday,23,143,0.066698,2,False,False,False,False,-74.000954,40.742031,-73.947708,40.782200,9275.8,703.9,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1081
3,False,0,430,Saturday,23,143,0.018223,1,False,False,False,False,-73.985733,40.738258,-73.993179,40.754890,2726.9,298.0,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,800
4,False,0,430,Saturday,23,143,0.027198,1,False,False,False,False,-74.006615,40.740650,-73.985619,40.723362,3455.8,458.2,65,46,0.0,0.0,0.0,10.6,0.56,2.2,13.0,0.0,0.0,1151


In [29]:
df_copy.columns

Index(['is_hol', 'blizz', 'm_d', 'dow', 'hr', 'wkhr', 'L2', 'pas_cnt', 'p_JFK',
       'p_LGA', 'd_JFK', 'd_LGA', 'p_lng', 'p_lat', 'd_lng', 'd_lat',
       'total_dist', 'total_time', 'day_max_temp', 'day_min_temp',
       'day_rainfall', 'day_snowfall', 'day_snowdepth', 'hr_temp', 'hr_humid',
       'hr_dewpt', 'hr_windspeed', 'hr_gustspeed', 'hr_precip', 'dur'],
      dtype='object')

In [31]:
df_copy.to_csv('../_data/train__701778__all_vars.csv', index=False)